# Load models and get predictions

In [4]:
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from joblib import load
from tensorflow.keras.models import load_model

In [ ]:
# Define the feature names in the correct order
feature_names = [
    'age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 
    'cholesterol', 'fasting_blood_sugar', 'resting_electrocardiogram',
    'max_heart_rate_achieved', 'exercise_induced_angina', 'st_depression',
    'st_slope', 'num_major_vessels', 'thalassemia'
]
# Sample input 
sample_input = {
    'age': 55,
    'sex': 1,
    'chest_pain_type': 2,
    'resting_blood_pressure': 130,
    'cholesterol': 250,
    'fasting_blood_sugar': 0,
    'resting_electrocardiogram': 1,
    'max_heart_rate_achieved': 150,
    'exercise_induced_angina': 0,
    'st_depression': 1.0,
    'st_slope': 2,
    'num_major_vessels': 1,
    'thalassemia': 3
}

In [5]:
# lgbmclassifier model 
loaded_model = joblib.load('models/lgbm_model.pkl')

In [7]:
sample_input = {
    'age': 52,
        'sex': 1,
        'chest_pain_type': 2,
        'resting_blood_pressure': 128,
        'cholesterol': 205,
        'fasting_blood_sugar': 0,
        'resting_electrocardiogram': 1,
        'max_heart_rate_achieved': 184,
        'exercise_induced_angina': 0,
        'st_depression': 0.0,
        'st_slope': 2,
        'num_major_vessels': 0,
        'thalassemia': 2
}

In [8]:
# Define the expected feature names (must match X_train from training)
expected_features = [
    'age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol',
    'fasting_blood_sugar', 'resting_electrocardiogram', 'max_heart_rate_achieved',
    'exercise_induced_angina', 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia'
]

In [9]:
# Convert sample input to DataFrame with correct feature names
sample_df = pd.DataFrame([sample_input], columns=expected_features)

In [10]:
prediction = loaded_model.predict(sample_df)

In [11]:
prediction_proba = loaded_model.predict_proba(sample_df)

In [12]:
print(f"Prediction: {'Disease' if prediction[0] == 1 else 'No Disease'}")
print(f"Prediction Probability: {prediction_proba[0]}")

Prediction: Disease
Prediction Probability: [0.00215327 0.99784673]


In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from joblib import load
import os

class HeartDiseasePredictor:
    def __init__(self):
        # Initialize paths
        self.model_path = "models/computer_cnn_model.keras"
        self.scaler_path = "models/scaler.pkl"
        
        # Load assets
        self.model = self._load_model()
        self.scaler = self._load_scaler()
        self.feature_names = [
            'age', 'sex', 'chest_pain_type', 'resting_blood_pressure',
            'cholesterol', 'fasting_blood_sugar', 'resting_electrocardiogram',
            'max_heart_rate_achieved', 'exercise_induced_angina', 'st_depression',
            'st_slope', 'num_major_vessels', 'thalassemia'
        ]
    
    def _load_model(self):
        """Load the trained CNN model"""
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model file not found at {self.model_path}")
        
        try:
            return load_model(self.model_path)
        except Exception as e:
            raise RuntimeError(f"Error loading model: {str(e)}")
    
    def _load_scaler(self):
        """Load the saved scaler"""
        if not os.path.exists(self.scaler_path):
            raise FileNotFoundError(f"Scaler file not found at {self.scaler_path}")
        
        try:
            return load(self.scaler_path)
        except Exception as e:
            raise RuntimeError(f"Error loading scaler: {str(e)}")
    
    def preprocess_input(self, input_data):
        """Validate and preprocess input features"""
        # Convert to DataFrame to maintain feature order
        try:
            input_df = pd.DataFrame([input_data], columns=self.feature_names)
            
            # Convert all values to numeric
            input_df = input_df.apply(pd.to_numeric, errors='coerce')
            if input_df.isnull().any().any():
                raise ValueError("Invalid input values detected")
                
            return input_df.values
        except Exception as e:
            raise ValueError(f"Input processing failed: {str(e)}")
    
    def predict(self, input_features):
        """Make a prediction with confidence score"""
        try:
            # Preprocess
            X = self.preprocess_input(input_features)
            X_scaled = self.scaler.transform(X)
            X_scaled = X_scaled.reshape(1, 13, 1)  # Reshape for CNN
            
            # Predict
            probability = float(self.model.predict(X_scaled)[0][0])
            prediction = int(probability > 0.5)
            confidence = abs(probability - 0.5) * 2  # 0-1 range
            
            return {
                'probability': probability,
                'prediction': prediction,
                'confidence': confidence,
                'interpretation': 'Positive for heart disease' if prediction == 1 
                                 else 'Negative for heart disease'
            }
        except Exception as e:
            raise RuntimeError(f"Prediction failed: {str(e)}")

# Example Usage in Production
if __name__ == "__main__":
    # Initialize predictor
    try:
        predictor = HeartDiseasePredictor()
        print("✅ Predictor initialized successfully")
    except Exception as e:
        print(f"❌ Initialization failed: {e}")
        exit()

    # Sample input (would come from your API/UI in production)
    patient_data = {
        'age': 52,
        'sex': 1,
        'chest_pain_type': 2,
        'resting_blood_pressure': 128,
        'cholesterol': 205,
        'fasting_blood_sugar': 0,
        'resting_electrocardiogram': 1,
        'max_heart_rate_achieved': 184,
        'exercise_induced_angina': 0,
        'st_depression': 0.0,
        'st_slope': 2,
        'num_major_vessels': 0,
        'thalassemia': 2
    }

    # Make prediction
    try:
        result = predictor.predict(patient_data)
        print("\n🔍 Prediction Results:")
        print(f"- Probability: {result['probability']:.4f}")
        print(f"- Prediction: {result['interpretation']}")
        print(f"- Confidence: {result['confidence']:.1%}")
        
        # Clinical decision logic
        if result['prediction'] == 1:
            print("\n🚨 Clinical Alert: Consider further cardiac evaluation")
        else:
            if result['probability'] < 0.2:
                print("\n✅ Low Risk: Routine follow-up recommended")
            else:
                print("\n⚠️ Moderate Risk: Consider lifestyle counseling")
                
    except Exception as e:
        print(f"❌ Prediction error: {e}")

✅ Predictor initialized successfully
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step

🔍 Prediction Results:
- Probability: 0.9953
- Prediction: Positive for heart disease
- Confidence: 99.1%

🚨 Clinical Alert: Consider further cardiac evaluation
